# **Ejercicios de pair programming Módulo 3 Sprint 1**
## **Regreción Lineal: Lección 7 - ANOVA**

En el pair programming de hoy usaremos el set de datos que guardastéis en el pair programming de normalización y estandarización.


Hasta ahora habéis estado evaluando las características de vuestro set de datos y habéis hecho una gran exploración, es el momento de hacer vuestro primer ANOVA! En el ejercicio de hoy tendréis que hacer un ANOVA con vuestro datos y hacer una interpretación de los resultados.


📌 NOTA Puede que vuestros datos no se ajusten o no cumplan todas las asunciones, no pasa nada, haced el ANOVA e interpretad los resultados. En próximas lecciones aprenderemos que podemos hacer cuando nos encontramos en esta situación.

In [1]:

import numpy as np
import pandas as pd
import random 


import matplotlib.pyplot as plt
import seaborn as sns


import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.multivariate.manova import MANOVA
from sklearn.preprocessing import StandardScaler

plt.rcParams["figure.figsize"] = (10,8) 

In [5]:
df = pd.read_csv("datos/soci_econ_country_profiles_estandarizado.csv", index_col = 0)
df.sample(5)

,country,region,population_in_thousands_(2017),employment:_agriculture_(%_of_employed),employment:_industry_(%_of_employed),employment:_services_(%_of_employed),education:_government_expenditure_(%_of_gdp),quality_of_life_index,purchasing_power_index,safety_index,cost_of_living,consumer_price_index_(2010_=_100),human_capital_index_(hci)_(scale_0-1),"inflation,_consumer_prices_(annual_%)",urban_population_(%_of_total_population)_y,"population_age_distribution_(0-14_years,_%)","population_age_distribution_(60+_years,_%)",international_migrant_stock_(%_of_total_pop.),restaurant_price_index_normal
32,Lebanon,WesternAsia,-0.217704,0.182195,-0.201389,0.103300,-1.011765,-0.541327,-0.342801,-0.512806,0.343250,0.266357,-1.005935,1.056201,0.485067,0.717201,-0.694340,2.003945,0.423794
30,Japan,EasternAsia,1.849540,-0.190476,0.368056,0.126255,-0.541176,0.099884,0.673036,1.052748,1.027434,-0.478824,0.810089,-0.709266,0.597745,-0.472303,0.920755,-0.560158,0.303953
64,Venezuela (Bolivarian Republic of),SouthAmerica,0.223238,0.488613,0.409722,-0.361549,-2.334378,-2.189445,-1.386676,-2.513083,-0.332827,3.131649,-1.099875,2.590340,0.476143,1.241983,-0.852830,-0.331361,0.202571
7,Bulgaria,EasternEurope,-0.200625,0.041408,0.770833,-0.200861,-0.305882,-0.044315,-0.388701,-0.015506,-0.385233,-0.281770,-0.186944,0.022338,-0.014112,-0.320700,0.490566,-0.575937,0.127380
9,Chile,SouthAmerica,-0.013827,0.298137,-0.131944,-0.005739,0.070588,-0.075359,-0.183499,-0.420878,0.008541,0.591814,-0.198813,0.076556,0.451003,0.390671,-0.392453,-0.481262,0.270610


In [6]:
df.columns

Index(['country', 'region', 'population_in_thousands_(2017)',
       'employment:_agriculture_(%_of_employed)',
       'employment:_industry_(%_of_employed)',
       'employment:_services_(%_of_employed)',
       'education:_government_expenditure_(%_of_gdp)', 'quality_of_life_index',
       'purchasing_power_index', 'safety_index', 'cost_of_living',
       'consumer_price_index_(2010_=_100)',
       'human_capital_index_(hci)_(scale_0-1)',
       'inflation,_consumer_prices_(annual_%)',
       'urban_population_(%_of_total_population)_y',
       'population_age_distribution_(0-14_years,_%)',
       'population_age_distribution_(60+_years,_%)',
       'international_migrant_stock_(%_of_total_pop.)',
       'restaurant_price_index_normal'],
      dtype='object')

In [10]:
df.rename(columns = { 'population_in_thousands_(2017)': 'population_2017', 'employment:_agriculture_(%_of_employed)': 'employment_agriculture',  
                     'employment:_industry_(%_of_employed)':  'employment_industry', 'employment:_services_(%_of_employed)': 'employment_services', 
                     'education:_government_expenditure_(%_of_gdp)': 'education_government',
                     'consumer_price_index_(2010_=_100)' : 'consumer_price_index',
                     'human_capital_index_(hci)_(scale_0-1)': 'human_capital_index',
                     'inflation,_consumer_prices_(annual_%)': 'inflation',
                     'urban_population_(%_of_total_population)_y': 'urban_populatio',
                     'population_age_distribution_(0-14_years,_%)': 'population_age_distribution_0_14',
                     'population_age_distribution_(60+_years,_%)': 'population_age_distribution_65_mas',
                     'international_migrant_stock_(%_of_total_pop.)': 'international_migrant_stock',
                     'human_capital_index_(hci)_(scale_0-1)': 'human_capital_index'}, inplace = True)

In [14]:
df.head(5)

,country,region,population_2017,employment_agriculture,employment_industry,employment_services,education_government,quality_of_life_index,purchasing_power_index,safety_index,cost_of_living,consumer_price_index,human_capital_index,inflation,urban_populatio,population_age_distribution_0_14,population_age_distribution_65_mas,international_migrant_stock,restaurant_price_index_normal
0,Argentina,SouthAmerica,0.432582,-0.331263,0.131944,0.315638,0.258824,-0.022740,-0.243315,-1.258757,0.080347,-0.036551,-0.572700,0.374915,0.605509,0.927114,-0.437736,-0.307692,0.328501
1,Australia,Oceania,0.095085,-0.273292,-0.368056,0.487805,0.211765,0.550795,0.660990,-0.139554,0.893666,0.102195,0.566766,-0.030665,0.393466,0.239067,-0.015094,1.538462,0.587030
2,Austria,WesternEurope,-0.172512,-0.107660,0.243056,0.120516,0.352941,0.765464,0.530557,1.143569,0.629026,0.018589,0.507418,0.030085,-0.615413,-0.332362,0.294340,0.694280,0.488644
3,Belarus,EasternEurope,-0.160047,0.298137,1.131944,-0.527977,0.070588,-0.338766,-0.699621,0.976879,-0.461962,1.234337,-0.077916,1.839723,0.111590,-0.029155,0.007547,0.213018,0.205084
4,Belgium,WesternEurope,-0.126655,-0.397516,-0.368056,0.573888,0.870588,0.302134,0.434187,-0.255296,0.740210,-0.024568,0.293769,0.050562,0.830865,0.017493,0.256604,0.284024,0.637479


In [16]:
# iniciamos nuestro anova. Recrestaurant_price_index_normalordemos que la variable respuesta es el precio de las casas (median_house_value)
lm = ols('''restaurant_price_index_normal ~ region + population_2017 + employment_agriculture +
       employment_industry + employment_services + education_government +
       quality_of_life_index + purchasing_power_index + safety_index +
       cost_of_living + consumer_price_index + human_capital_index +
       inflation + urban_populatio + population_age_distribution_0_14 +
       population_age_distribution_65_mas + international_migrant_stock''', 
         data=df).fit()
sm.stats.anova_lm(lm)

,df,sum_sq,mean_sq,F,PR(>F)
region,13.0,2.445509e+00,1.881161e-01,62.583515,1.058411e-20
population_2017,1.0,3.395810e-02,3.395810e-02,11.297371,1.849431e-03
employment_agriculture,1.0,2.899746e-01,2.899746e-01,96.470375,1.001780e-11
employment_industry,1.0,4.587495e-02,4.587495e-02,15.261935,3.955311e-04
employment_services,1.0,5.290443e-02,5.290443e-02,17.600543,1.698396e-04
education_government,1.0,1.206160e-03,1.206160e-03,0.401272,5.304371e-01
quality_of_life_index,1.0,3.662014e-04,3.662014e-04,0.121830,7.290915e-01
purchasing_power_index,1.0,2.450878e-02,2.450878e-02,8.153719,7.089824e-03
safety_index,1.0,2.425718e-03,2.425718e-03,0.807001,3.749775e-01
cost_of_living,1.0,7.166159e-01,7.166159e-01,238.407819,1.870293e-17


In [17]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                  OLS Regression Results                                 
=========================================================================================
Dep. Variable:     restaurant_price_index_normal   R-squared:                       0.971
Model:                                       OLS   Adj. R-squared:                  0.948
Method:                            Least Squares   F-statistic:                     41.85
Date:                           Tue, 09 May 2023   Prob (F-statistic):           4.02e-20
Time:                                   12:11:31   Log-Likelihood:                 117.99
No. Observations:                             66   AIC:                            -176.0
Df Residuals:                                 36   BIC:                            -110.3
Df Model:                                     29                                         
Covariance Type:                       nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                             -0.1500      0.446     -0.336      0.739      -1.055       0.755
region[T.EasternAsia]                 -0.2417      0.086     -2.824      0.008      -0.415      -0.068
region[T.EasternEurope]                0.0714      0.075      0.950      0.348      -0.081       0.224
region[T.NorthernAfrica]               0.1945      0.134      1.455      0.154      -0.077       0.466
region[T.NorthernAmerica]              0.0421      0.085      0.496      0.623      -0.130       0.214
region[T.NorthernEurope]               0.1440      0.075      1.910      0.064      -0.009       0.297
region[T.Oceania]                      0.0288      0.082      0.349      0.729      -0.138       0.196
region[T.South-easternAsia]           -0.1822      0.079     -2.306      0.027      -0.342      -0.022
region[T.SouthAmerica]                -0.0594      0.067     -0.883      0.383      -0.196       0.077
region[T.SouthernAfrica]               0.1683      0.106      1.591      0.120      -0.046       0.383
region[T.SouthernAsia]                 0.0422      0.086      0.490      0.627      -0.132       0.217
region[T.SouthernEurope]               0.0615      0.079      0.777      0.442      -0.099       0.222
region[T.WesternAsia]                  0.0211      0.068      0.310      0.758      -0.117       0.159
region[T.WesternEurope]                0.1055      0.081      1.308      0.199      -0.058       0.269
population_2017                        0.0047      0.003      1.785      0.083      -0.001       0.010
employment_agriculture                 1.8673      2.171      0.860      0.395      -2.536       6.270
employment_industry                    1.0881      1.291      0.843      0.405      -1.531       3.707
employment_services                    2.6624      3.129      0.851      0.400      -3.684       9.008
education_government                   0.0005      0.001      0.534      0.597      -0.001       0.002
quality_of_life_index                 -0.0018      0.036     -0.050      0.960      -0.076       0.072
purchasing_power_index                -0.0692      0.033     -2.087      0.044      -0.136      -0.002
safety_index                          -0.0051      0.021     -0.249      0.805      -0.047       0.037
cost_of_living                         0.3630      0.027     13.684      0.000       0.309       0.417
consumer_price_index                  -0.0086      0.009     -0.938      0.354      -0.027       0.010
human_capital_index                    0.0667      0.036      1.869      0.070      -0.006       0.139
inflation                             -0.0076      0.011     -0.671      0.506

- El R-Square de nuetros datos es muy alto, llegando a 0.971, es decir, los datos sugieren que nuestro modelo de predición es muy fiable porque explica el 97.1% de la variación de la variable respuesta. Sin embargo, estas conclusiones son fictícias ya que nuestros datos no siguen una distribución normal. 
  
- Las variables que afectan la variable respuesta (pvalue <= 0.05) en nuetros datos son: 'region', 'purchasing_power_index', 'cost_of_living', 'urban_populatio', 'international_migrant_stock'. 
  
- De las columnas que afectan nuestra VR: 'region' (con distintos valores), 'purchasing_power_index' (coef: -0.0692), 'cost_of_living' (coef 0.3630),  'urban_populatio' (coef 0.0714), 'international_migrant_stock' (-0.0125). En el caso de la variable 'purchasing_power_index' (coef: -0.0692), por cada unidad que aumenta 'purchasing_power_index', la variable respuesta disminuye 0.0692. 